<a href="https://colab.research.google.com/github/sdgroeve/ML-course-VIB-2020/blob/master/Histone_marks_dt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Histone modifications

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

random_seed = 123
np.random.seed(random_seed)

# 1. Reading the data

In [12]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_test.csv")
test_labels = pd.read_csv("data_test_label.csv")

In [13]:
test["Label"] = test_labels["Label"]

In [14]:
data = pd.concat([train,test])

In [15]:
data.shape

(15485, 502)

In [16]:
data = data.sample(frac=1)

In [19]:
train = data.iloc[0:3000,:]
test = data.iloc[3000:,:]

In [20]:
train

,GeneId,H3K4me3_0,H3K4me1_0,H3K36me3_0,H3K9me3_0,H3K27me3_0,H3K4me3_1,H3K4me1_1,H3K36me3_1,H3K9me3_1,...,H3K4me1_98,H3K36me3_98,H3K9me3_98,H3K27me3_98,H3K4me3_99,H3K4me1_99,H3K36me3_99,H3K9me3_99,H3K27me3_99,Label
8425,6710,2,1,2,0,2,3,1,1,0,...,0,0,0,0,1,1,1,1,1,0
9553,2217,1,3,1,1,0,2,0,1,0,...,0,0,1,1,0,0,1,0,0,0
2764,10302,1,9,4,4,0,0,5,1,5,...,0,0,0,0,0,0,0,0,1,1
3616,11952,0,3,13,0,1,1,2,14,0,...,3,0,3,0,1,1,0,1,0,1
2327,11551,0,3,0,1,0,1,2,0,1,...,0,0,0,0,0,1,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3492,1512,0,1,4,3,1,0,5,3,3,...,2,1,1,0,1,1,0,1,1,1
4883,4641,5,2,3,2,3,4,1,2,2,...,1,2,2,1,1,3,3,2,0,1
4508,12640,4,2,1,0,0,2,3,1,2,...,1,3,3,0,3,3,8,2,1,0
6856,1472,0,3,4,3,1,0,3,5,3,...,0,0,1,1,0,1,0,0,1,1


In [21]:
train_index_col = train.pop("GeneId")

In [22]:
test_index_col = test.pop("GeneId")

In [23]:
from pycaret.classification import *

In [7]:
#models()

In [24]:
setup(data=data,target="Label",numeric_features=list(data.columns)[:-1], 
      train_size=0.8, silent=True, use_gpu=False)

,Description,Value
0,session_id,2542
1,Target,Label
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15485, 502)"
5,Missing Values,False
6,Numeric Features,501
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


(       GeneId  H3K4me3_0  H3K4me1_0  H3K36me3_0  H3K9me3_0  H3K27me3_0  \
 4895   2730.0        1.0        1.0         0.0        2.0         0.0   
 142    5352.0        1.0        0.0         3.0        0.0         1.0   
 5029   4996.0        5.0        4.0         8.0        3.0         2.0   
 4201  13867.0        1.0        3.0         2.0        1.0         0.0   
 8746  12239.0        1.0        2.0         1.0        0.0         2.0   
 ...       ...        ...        ...         ...        ...         ...   
 7870   7708.0        0.0        1.0         0.0        0.0         0.0   
 8628  11434.0        1.0        0.0         0.0        1.0         1.0   
 909    8966.0        1.0        5.0         7.0        1.0         5.0   
 2822  10810.0        1.0        2.0         2.0        1.0         0.0   
 2416  13731.0        2.0        1.0         5.0        7.0         0.0   
 
       H3K4me3_1  H3K4me1_1  H3K36me3_1  H3K9me3_1  ...  H3K4me3_98  \
 4895        3.0        0.0

In [25]:
from sklearn.metrics import log_loss

add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function               <function log_loss at 0x0000019AA1FCBEE0>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [28]:
compare_models(cross_validation=False,sort="LogLoss")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
catboost,CatBoost Classifier,0.8618,0.9306,0.8628,0.8525,0.8576,0.7234,0.7234,0.3349,30.6000
lightgbm,Light Gradient Boosting Machine,0.8589,0.9256,0.8594,0.8498,0.8546,0.7176,0.7176,0.3460,1.6800
gbc,Gradient Boosting Classifier,0.8531,0.9252,0.8568,0.8416,0.8491,0.7060,0.7061,0.3486,42.1600
rf,Random Forest Classifier,0.8553,0.9241,0.8635,0.8409,0.8520,0.7106,0.7108,0.3652,2.5400
et,Extra Trees Classifier,0.8608,0.9215,0.8728,0.8440,0.8582,0.7216,0.7220,0.3829,1.9900
lr,Logistic Regression,0.8469,0.9091,0.8447,0.8391,0.8419,0.6936,0.6936,0.3923,4.9200
lda,Linear Discriminant Analysis,0.8440,0.9104,0.8307,0.8436,0.8371,0.6875,0.6876,0.3955,1.0700
xgboost,Extreme Gradient Boosting,0.8621,0.9236,0.8621,0.8535,0.8578,0.7240,0.7240,0.3973,7.7300
ada,Ada Boost Classifier,0.8489,0.9127,0.8414,0.8448,0.8431,0.6974,0.6974,0.6741,7.4600
knn,K Neighbors Classifier,0.8072,0.8602,0.7450,0.8375,0.7885,0.6125,0.6161,2.1628,2.0400


In [35]:
from pycaret.internal.distributions import *

tune_distributions = {
            "num_leaves": IntUniformDistribution(10, 20),
            "learning_rate": UniformDistribution(0.000001, 0.5, log=True),
            "n_estimators": IntUniformDistribution(10, 300),
            "min_split_gain": UniformDistribution(0, 1),
            "reg_alpha": UniformDistribution(0.0000000001, 10, log=True),
            "reg_lambda": UniformDistribution(0.0000000001, 10, log=True),
            "min_data_in_leaf": IntUniformDistribution(10, 10000),
            "feature_fraction": UniformDistribution(0.4, 1),
            "bagging_fraction": UniformDistribution(0.4, 1),
            "bagging_freq": IntUniformDistribution(1, 7),
            "min_child_samples": IntUniformDistribution(5, 100),
        }

In [38]:
lightgbm = create_model("lightgbm")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.8555,0.9187,0.8770,0.8433,0.8598,0.7109,0.7115,0.3641
1,0.8507,0.9166,0.8514,0.8528,0.8521,0.7013,0.7013,0.3663
2,0.8571,0.9204,0.8530,0.8627,0.8578,0.7143,0.7143,0.3611
3,0.8200,0.8969,0.8387,0.8114,0.8248,0.6399,0.6402,0.4148
4,0.8467,0.8974,0.8387,0.8550,0.8468,0.6933,0.6935,0.4066
5,0.8555,0.9199,0.8581,0.8567,0.8574,0.7110,0.7110,0.3590
6,0.8547,0.9164,0.8852,0.8371,0.8605,0.7092,0.7104,0.3650
7,0.8709,0.9260,0.8596,0.8822,0.8708,0.7418,0.7420,0.3427
8,0.8554,0.9130,0.8610,0.8542,0.8576,0.7108,0.7108,0.3730
9,0.8522,0.9322,0.8562,0.8521,0.8542,0.7043,0.7043,0.3343


In [47]:
from pycaret.internal.utils import (
    param_grid_to_lists,
    get_logger,
    get_class_name,
    np_list_arange,
)

tune_grid = {
            "num_leaves": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200],
            "learning_rate": np_list_arange(0.001, 0.5, 0.001, inclusive=True),
            "n_estimators": np_list_arange(10, 300, 10, inclusive=True),
            "min_split_gain": [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
            "reg_alpha": [
                0.0000001,
                0.000001,
                0.0001,
                0.001,
                0.01,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.15,
                0.2,
                0.3,
                0.4,
                0.5,
                0.7,
                1,
                2,
                3,
                4,
                5,
                10,
            ],
            "reg_lambda": [
                0.0000001,
                0.000001,
                0.0001,
                0.001,
                0.01,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.15,
                0.2,
                0.3,
                0.4,
                0.5,
                0.7,
                1,
                2,
                3,
                4,
                5,
                10,
            ],
            "feature_fraction": np_list_arange(0.4, 1, 0.1, inclusive=True),
            "bagging_fraction": np_list_arange(0.4, 1, 0.1, inclusive=True),
            "bagging_freq": [1, 2, 3, 4, 5, 6, 7],
            "min_child_samples": np_list_arange(5, 100, 5, inclusive=True),
        }

tuned_lightgbm = tune_model(lightgbm, optimize="LogLoss", tuner_verbose=1, n_iter=10,
                            custom_grid=tune_grid)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.8612,0.9188,0.8770,0.8525,0.8646,0.7222,0.7225,0.3604
1,0.8579,0.9198,0.8578,0.8606,0.8592,0.7159,0.7159,0.3588
2,0.8523,0.9211,0.8546,0.8533,0.8540,0.7046,0.7046,0.3569
3,0.8273,0.9006,0.8498,0.8160,0.8326,0.6544,0.6549,0.4013
4,0.8450,0.9023,0.8403,0.8511,0.8457,0.6901,0.6901,0.3917
5,0.8636,0.9242,0.8660,0.8646,0.8653,0.7272,0.7272,0.3487
6,0.8596,0.9207,0.8947,0.8386,0.8657,0.7189,0.7205,0.3553
7,0.8692,0.9280,0.8628,0.8768,0.8698,0.7385,0.7386,0.3400
8,0.8522,0.9171,0.8626,0.8477,0.8551,0.7043,0.7044,0.3625
9,0.8530,0.9348,0.8562,0.8535,0.8549,0.7059,0.7059,0.3348


# 2. Fitting a decision tree model

The scikit-learn `DecisionTreeClassifier` class computes a decision tree predictive model from a dataset. 

To get all the options for learning you can simply type: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
help(DecisionTreeClassifier)

You notice that there are many (hyper)parameters to set. These influence the complexity of the model. An important such parameter is the `max_depth` that sets a limit on how deep a decision tree can become. 

Let's create a decision tree model with `max_depth=3`:

In [ ]:
cls = DecisionTreeClassifier(max_depth=3)

This creates a decision tree model with default values for the other hyperparameters:

In [ ]:
cls

Let's create a validation set, fit the model and evaluate.

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                  test_size=.2, random_state=random_seed)

cls.fit(train_X,train_y)

predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)

print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)

print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,-1]),log_loss(val_y,predictions_val_prob[:,-1])))

The following code plots the fitted decision tree `cls` as a `tree.png` file:

In [ ]:
"""
from sklearn import tree
from io import StringIO
from IPython.display import Image, display
import pydotplus

out = StringIO()
tree.export_graphviz(cls_DT, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png("tree.png")
"""

How do other values for for the `max_depth` hyperparameter perform?

In [ ]:
import seaborn as sns

result = []
for md in range(1,10):
  cls = DecisionTreeClassifier(max_depth=md)
  cls.fit(train_X,train_y)
  predictions_train_prob = cls.predict_proba(train_X)
  predictions_val_prob = cls.predict_proba(val_X)
  result.append([md,log_loss(train_y,predictions_train_prob[:,-1]),"train"])
  result.append([md,log_loss(val_y,predictions_val_prob[:,-1]),"val"])

toplot = pd.DataFrame(result,columns=["max_depth","log-loss","set"])
sns.lmplot(x="max_depth",y="log-loss",hue="set",data=toplot,fit_reg=False)

In [ ]:
cls = DecisionTreeClassifier(max_depth=14)

predictions_list = []
for i in range(10):
  train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                    test_size=.2, random_state=i)

  cls.fit(train_X,train_y)
  predictions_val = cls.predict(val_X)
  predictions_val_prob = cls.predict_proba(val_X)
  predictions_list.append(list(predictions_val_prob[:,-1]))
  print("%f %f"%(log_loss(val_y,predictions_val_prob[:,-1]),accuracy_score(val_y,predictions_val)))

In [ ]:
tmp = pd.DataFrame(predictions_list)
predictions_avg = tmp.mean(axis=0)
print("Avg. model: %f"%(log_loss(val_y,predictions_avg)))

In [ ]:

#pd.plotting.scatter_matrix(tmp.transpose())
sns.pairplot(tmp.transpose(),kind="scatter")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

scaler_minmax = preprocessing.MinMaxScaler()
scaler_minmax.fit(train)
train_norm = pd.DataFrame(scaler_minmax.transform(train),columns=train.columns)

#cls = DecisionTreeClassifier(max_depth=4)
cls = LogisticRegression(C=0.5)

predictions_list = []
for i in range(10):
  train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                    test_size=.2, random_state=i)

  cls.fit(train_X,train_y)
  predictions_val = cls.predict(val_X)
  predictions_val_prob = cls.predict_proba(val_X)
  predictions_list.append(list(predictions_val_prob[:,-1]))
  print("%f %f"%(log_loss(val_y,predictions_val_prob[:,-1]),accuracy_score(val_y,predictions_val)))

In [ ]:
tmp = pd.DataFrame(predictions_list)
#pd.plotting.scatter_matrix(tmp.transpose())
sns.pairplot(tmp.transpose(),kind="scatter")

In [ ]:
predictions_avg = tmp.mean(axis=0)
print("Avg. model: %f"%(log_loss(val_y,predictions_avg)))

# 5. Hyperparamters

For our first submission we set the hyperparameter `max_depth=3`. Other values might result in lower log-loss on the testset. 

Since we don't have the testset labels we can only check this on the public leaderboard, which we can/should not do!

So, we need to create our own testset (**not seen during training!**) with known class labels.

Scikit-learn offers many options to do this. One of them is the `train_test_split` function:

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.2, random_state=random_seed)

#train fold
print(train_X.shape)
print(train_y.shape)
#validation fold
print(val_X.shape)
print(val_y.shape)

Fit a decision tree model with `max_depth=14` default paramters on the `train_X` data set.

In [ ]:
#solution
cls_DT = DecisionTreeClassifier(max_depth=14)
cls_DT.fit(train_X,train_y)

What is the accuracy and log-loss on `train_X`? 

In [ ]:
#solution
predictions = cls_DT.predict(train_X)
print("Accuracy: %f"%(accuracy_score(predictions, train_y)))
predictions = cls_DT.predict_proba(train_X)
print("Log-loss: %f"%(log_loss(train_y,predictions[:,1])))

What is the accuracy and log-loss on `val_X`?

In [ ]:
#solution
predictions = cls_DT.predict(val_X)
print("Accuracy: %f"%(accuracy_score(predictions, val_y)))
predictions = cls_DT.predict_proba(val_X)
print("Log-loss: %f"%(log_loss(val_y,predictions[:,1])))

What do you see?


The following code evaluates different values for this hyperparameter.

In [ ]:
for maxdepth in range(1,20,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    cls.fit(train_X,train_y)
    predictions_train = cls.predict(train_X)
    predictions_val = cls.predict(val_X)
    predictions_train_prob = cls.predict_proba(train_X)[:,1]
    predictions_val_prob = cls.predict_proba(val_X)[:,1]
    print("%i (%f) %f (%f) %f"%(maxdepth,
                                accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y),
                                log_loss(train_y,predictions_train_prob),log_loss(val_y,predictions_val_prob)))

What do you see?

So, we have split the data into a train- and validationset. We can of course split the data in many different ways (different random seeds) resulting in different train- and validationsets.

Let's try 5 different random seeds:

In [ ]:
for run in range(5):
  train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.8, random_state=run)
  min_m = 100
  best = None
  for maxdepth in range(1,20,1):
      cls = DecisionTreeClassifier(max_depth=maxdepth)
      cls.fit(train_X,train_y)
      predictions_val_prob = cls.predict_proba(val_X)[:,1]
      m = log_loss(val_y,predictions_val_prob)
      if m < min_m:
        min_m = m
        best = maxdepth
  print("%i %f"%(best,min_m))

What do you see?

The solution is to run several train-validations splits and average the performance.

One popular method is cross-validation that uses each datapoint once as a testpoint.

It works as follows:
<br/>
<br/>
<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png"/>
<br/>
<br/>

It is easy to run this in Scikit-learn:



In [ ]:
from sklearn.model_selection import cross_val_predict

for maxdepth in range(1,10,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    predictions = cross_val_predict(cls,train_norm,train_labels,
                                    cv=10,
                                    method="predict_proba")
    print("%i %f"%(maxdepth,log_loss(train_labels,predictions[:,1])))

We can do this in two lines of code with the `GridSearchCV` module:


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':range(1,10)
    }

GSCV = GridSearchCV(cls_DT, params,
                    cv=10,
                    scoring="neg_log_loss",
                    verbose=1).fit(train_norm,train_labels)

print(GSCV.best_estimator_)
print(GSCV.best_score_)

Play with the hyperparameters in a Template notebook and make some Kaggle submissions.

# 5. Ensemble learning: bagging

We have seen that bias and variance play an important role in Machine Learning. 

Let's first see what bagging can do for our dataset. 

In [ ]:
from sklearn.ensemble import BaggingClassifier

cls = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),random_state=random_seed)
                                                            
cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

With the `RandomForestClassifier` the variance of the decision tree is reduced also by selecting features for decision tree contruction at random. Let's see how far we get with default hyperparameter values.   

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(random_state=random_seed)

cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

# 6. Ensemble learning: boosting

How about the `GradientBoostingClassifier`?

In [ ]:
#solution
from sklearn.ensemble import GradientBoostingClassifier

cls = GradientBoostingClassifier(random_state=random_seed,
                                    max_depth=10)
cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))